# How to execute script in singularity

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:

  

Before you run any script using singularity, please check if `singularity` is installed by checking the availability of command `singularity`. Also, it would be helpful for you to read the [sigularity user's guide](https://www.sylabs.io/docs/) or some online tutorial to understand how singularity works before you try to use singularity in SoS.

Although running singularity images does not need root privilege, building singularity images often requires `sudo` access. SoS provides an option `sudo=True` to the `singularity_build` action (an equivalence to command `singularity build`) to execute the command in sudo mode, but it does not accept interactive input of password. So before running any `singularity_build` action with option `sudo=True`, please run `sudo -i` to enter sudo mode, or add your username as a sudo user without password (google "sudo without password" for instructions). The latter is not safe but can be very convenient for you to prepare singularity images on a personal linux workstation.

## Running a script using singularity

SoS executes scripts with a singularity container by calling command `singularity exec` with appropriate parameters. For example, if you specify a container with a `shub:` schema, sos will first pull the image, save it as a local image, and use `singularity exec` to run it:

In [1]:
%run
run: container='shub://vsoch/singularity-hello-world:master'
  cat /etc/os-release

INFO: Running default :

HINT: Pulling image shub://vsoch/singularity-hello-world:master to vsoch-singularity-hello-world-master.simg


WARNING: WARNING client.py:242 - Failed to pull shub://vsoch/singularity-hello-world:master: FATAL: While pulling shub image: failed to get manifest for: shub://vsoch/singularity-hello-world:master: 502 Bad Gateway

ERROR: ERROR step_executor.py:1743 - default (id=1247662052776750943) returns an error. - None

ERROR: [default]: [0]: 
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
script_3016194913819825155 in <module>
----> run('cat /etc/os-release\n', container='shub://vsoch/singularity-hello-world:master')
      

ValueError: Image vsoch-singularity-hello-world-master.simg does not exist after pulling shub://vsoch/singularity-hello-world:master.


The actual `singularity exec` command executed by SoS can be shown when you execute the script in dryrun mode (with `-n` option). In this mode, SoS would print the interpolated script (if option `expand=True` is set) and the docker command to execute it:

In [2]:
%rerun -n

File contains parsing errors: 
	[line  2]: %rerun -n

Invalid statements: SyntaxError('invalid syntax', ('<string>', 1, 1, '%rerun -n\n'))


As you can see, the docker command looks similar to

```
singularity exec singularityhub-ubuntu.simg /bin/bash -ev /path/to/a/temp/script
```

Basically, SoS pulls the image and runs command `singularity exec` to execte the script with `/bin/bash` with the singularity image.

You can also use a docker image with singularity. However, because `docker://` images are by default executed by docker, you will need to specify the use of singularity using parameter `engine='singularity'`:

In [ ]:
%run
run: container='docker://ubuntu', engine='singularity'
  cat /etc/os-release

In summary, as listed [here](https://vatlab.github.io/sos-docs/doc/documentation/Targets_and_Actions.html#container-and-engine-124), you can invoke `singularity` with the following combinations of parameters `container` and `engine`:

| `container` | `engine` | execute by | example | comment | 
| -- | -- | -- | -- | -- |
| `filename.simg` | ` ` | singularity | `container='ubuntu.simg'` | |
| `shub://tag` | ` ` | singularity | `container='shub://GodloveD/lolcow'` | Image will be pulled to a local image |
| `name` | `singularity` | singularity | `container='a_dir', engine='singularity'` | treat `name` as singularity image file or directory |
| `docker://tag` | `singularity` | singularity |  `container='docker://godlovdc/lolcow', engine='singularity'`  |  |
| `file://filename` | ` ` | singularity | `container='file://ubuntu.simg'` | |

Simply put, singularity will be used by default with container `shub://`, `file://`, and `filename.simg`, but you will have to specify `engine='singularity'` if you would like to use a docker image or a directory.

## Building singularity images

### Action `singularity_build` <a id="singularity_build"></a>

Action `singularity_build` calls command `singularity build` with appropriate command line options. The SoS equivalence of the example in the singularity user's guide

```
singularity build lolcow.simg shub://GodloveD/lolcow
```
is the `singularity_build` action with options `src` and `dest`:

In [ ]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow.simg')

As you can see from the warning message, you are recommended to build the image with root privilege. In this case, you can add option `sudo=True` when you are certain that the user is in SUDO (no password) mode:

In [ ]:
singularity_build(src='shub://GodloveD/lolcow', dest='lolcow_sudo.simg', sudo=True)

You can also use action `singularity_build` to build an image from a singularity definition file. Using an example from the user's guide:

In [ ]:
singularity_build: dest='lolcow.simg', sudo=True
    Bootstrap: docker
    From: ubuntu:16.04
    %post
        apt-get -y update
        apt-get -y install fortune cowsay lolcat
    %environment
        export LC_ALL=C
        export PATH=/usr/games:$PATH
    %runscript
        fortune | cowsay | lolcat 

Options such as `notest=True` could be add to the action. Note that the content of the definition file is indented for  clarify, but you can include the file as it is (no indentation).

## Running script with singularity image

Although singularity accepts `docker://` and `shub://` container tags, SoS always pull the image and build a local `.simg` file before executing it. If the container is used again, the local `simg` file will be used directly.

For example, the following action will pull `docker://ubuntu` and create `ubuntu.simg`

In [ ]:
run: container='docker://ubuntu', engine='singularity'
  ls /

and you can use `container='ubuntu.simg'` directly if you have an exiting `ubuntu.simg` file

In [ ]:
run: container='ubuntu.simg'
  ls /

### Binding directories (option `bind`)

A very useful feature about singularity is that you can use the container almost as a local command with access to local file system,

In [ ]:
run: container='ubuntu.simg'
  wc -l ~/.bashrc

However, singularity only binds current working directory, `/tmp`, and your home directory. Other directories would be from within the image, or appear to be missing even if they exist on the host file system. For example, the following command lists `/usr/local` inside the image:

In [ ]:
run: container='ubuntu.simg'
  ls /usr/local

and `/usr/local/var` would appear to be missing

In [ ]:
run: container='ubuntu.simg'
  ls /usr/local/var

To allow singularity to see more directories, you can add one or more parameters to the `bind` parameter. For example, with `bind='/usr/local'`, the `singularity exec` command lists directory `/usr/local` from the host filesystem:

In [ ]:
run: container='ubuntu.simg', bind='/usr/local'
  ls /usr/local

and we can see `/usr/local/var` actually exists

In [ ]:
run: container='ubuntu.simg', bind='/usr/local'
  ls /usr/local/var

Parameter `bind` accepts `host_dir:img_dir` pairs to mount `host_dir` from host as `img_dir` seen by the image

In [ ]:
%run -v3
run: container='ubuntu.simg', bind='/usr/local/var:/myvar'
  ls /myvar